# 10. Lab10_NN, ReLU, Xavier, Dropout, and Adam
### 2018.09.28(금)
> Neural Net을 사용하면서 굉장히 좋은 TIP 들

### Remind
Softmax classifier for MNIST
```
tf.nn.softmax_cross_entropy_with_logits(logits = hypothesis, labels = Y)
```

> LAB7-2 MNIST data 참고

In [1]:
import tensorflow as tf
import numpy as np

/Users/Choi-seonyeol/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## NN for MNIST
__IDEA01 : Layer를 여러개를 해서 예측을 하면 성능이 더 좋다! __

In [3]:
# input place holders
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

# weights & bias for nn Layers
W1 = tf.Variable(tf.random_normal([784, 256]))
b1 = tf.Variable(tf.random_normal([256]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([256, 256]))
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.Variable(tf.random_normal([256, 10]))
b3 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2, W3) + b3

# define cost/Loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = hypothesis, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

NameError: name 'learning_rate' is not defined

## Xavier for MNIST
__IDEA02 : W의 초기값을 잘 정하면 성능이 더 좋다!__

1. Xavier initialization : 몇개의 input, 몇개의 output인지에 따라 이에 비례해서 weight를 결정 
2. He's initialization : Xavier 를 개선 fan_in을 2로 나누면 성능 개선

~~~
# Xavier initialization
W = np.random.randn(fan_in, fan_out)/np.sqrt(fan_in)

즉, in의 개수보다 크고 out의 개수보다 작은 random 값을 root(in의 개수) 로 나누어준 값을 W로 쓰면 좋다.

# He's
W = np.random.randn(fan_in, fan_out)/np.sqrt(fan_in/2)

즉,Xavier의 Initialization 에서, root(in의 개수) 가 아니라 root(in의 개수 / 2)로 나누어주면 성능이 더 좋아진다. 
~~~

Xavier를 적용시킨 후 결과값을 확인해보면, 처음부터 cost가 낮게 시작된다.
이는 곧, W의 초기값이 아주 잘 선택되었다는 것을 알 수있다. (심지어, 쓰지 않았을때 15 epoch만큼 실행시켰을 때보다 Cost가 낮다.)

In [ ]:
# input place holders
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

# Weight & bias for nn Layers
# http://stackoverflow.com/questions/33640581
W1 = tf.get_variable("W1",shape = [784, 256], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([256]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.get_variable("W2",shape = [256, 256], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.get_variable("W3",shape = [256, 10], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2, W3) + b3

# define cost/Loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis,labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


## Deep NN for MNIST ( & wide)
__IDEA03 : 여기서 멈추지 않고, Layer의 수를 5개까지 확장, node의 수를 512개로 확장__

BUT, 결과는 오히려 더 낮아진다.Accuracy 0.9742 로 떨어짐.
아마, 이것은 overfitting의 결과로 보임 ( 지나친 deep 으로, Training data에 지나치게 최적화)

--> 이를 해결하기 위해, 우리는 Dropout을 배웠다.

In [ ]:
# input place holders
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

# weights & bias for nn layers
# http://stackoverflow.com/questions/33640581/how-to-do-xavier-initialization-on-tensorflow
W1 = tf.get_variable("W1", shape=[784, 512],initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.get_variable("W2", shape=[512, 512],initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([512]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.get_variable("W3", shape=[512, 512],initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([512]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)

W4 = tf.get_variable("W4", shape=[512, 512],initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([512]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)

W5 = tf.get_variable("W5", shape=[512, 10],initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L4, W5) + b5

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

### Dropout for MNIST
__IDEA04 : node 몇몇개는 쉬게해서 overfitting 을 막는다.__

__Deep 하게 했더니, 오히려 정확도가 떨어졌다. 이는 overfitting 이 원인으로 보임__

__이를 해결하기 위해, node중 일부는 쉬게하는 Dropout을 활용__

In [ ]:
# input place holders
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

# dropout (keep_prob) rate  0.7 on training, but should be 1 for testing
keep_prob = tf.placeholder(tf.float32)

# weights & bias for nn layers
# http://stackoverflow.com/questions/33640581/how-to-do-xavier-initialization-on-tensorflow
W1 = tf.get_variable("W1", shape=[784, 512],initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob) ## dropout하는 부분!

W2 = tf.get_variable("W2", shape=[512, 512],initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([512]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob) ## dropout하는 부분!

W3 = tf.get_variable("W3", shape=[512, 512],initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([512]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob) ## dropout하는 부분!

W4 = tf.get_variable("W4", shape=[512, 512],initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([512]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob) ## dropout하는 부분!

W5 = tf.get_variable("W5", shape=[512, 10],initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L4, W5) + b5

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train my model
for epoch in range(training_epochs):
    avg_cost = 0

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.7} ## 실행할때 dropout 0.7로 설정! -> 몇 %의 node를 keep할것인가ㅠ
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
    
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print('Accuracy:', sess.run(accuracy, feed_dict={
          X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1})) ## test를 할때는 dropout 하지 않고 모든 node를 사용

## Optimizer
우리는 여태까지 GradientDescentOptimizer를 사용했는데, 사실 optimizer 의 종류는 정말 다양하다.
여러가지 optimizer 를 사용해보며 더 높은 성능을 내는 optimizer를 사용하자.
![lec10_11](../../img/lec10_17.png)

지금까지는 Adam 이 cost를 빨리 줄인다.
그래서, 통상적으로 Adam 으로 시작하면 좋다.

### Use Adam optimizer
~~~
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = hypothesis, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
~~~

![summary](../../img/lec10_18.png)